# Tutorial for `musiF`

`musiF` is a python library to analyse music scores. It is a tool to massively extract features from MusicXML and Musescore files. 

`musiF` was born in the context of the [Didone Project](https://didone.eu/) and, consequently,
it is specialized in a 18th Century Italian Operas arias. However, it is likely helpful for the analysis intended to work in other repertoires as well.

## Installation

First, you should install Python > 3.10. We reccommend to use virtualenvs while you develop, so that you do not interfer with the other Python applications installed in your system.

An easy way to do this is by using [`anaconda`](https://www.anaconda.com/products/distribution), especially if you are not used to commandline interface. Alternatively, you can use more standard methods such Python's `virtualenv` module, `pyenv`, `poetry`, `pdm`, and similar.

In any case, to install, just use `pip install musif` from a command line inside your virtualenv.
You can also use a jupyer notebook or console and the `!` special command:

In [ ]:
!pip install -e /home/federico/musiF # TODO: change to pip install musif

Now, let's import it:

In [3]:
import musif
print(musif.__version__)

0.1.0


## Introduction

### If you are new to programming etc. read this

If you are new to Python, we suggest you to read a tutorial for it.
In the following, we will use some technical terminology. In general keep in mind the following:

* a _function_ is a way to represent code that is convenient for humans. You can think about functions as the mathematical functions, with some input and some output. However, some programming languages call them _procedures_; this is not the case of Python, but this name allow grasp what functions are, after all: they are successions of commands that the computer has to execute.

* an _object_ is a computational way to represent information in the memory of computers; you can think about objects as real concepts of the real world: object have properties (in Python named _fields_) and functionalities (named _methods_). For instance, an object could be a vehicle, which has some properties (length, maximum speed, number of wheels) and some functionalities (accelerate, decelerate, stop). Objects can also have specializations (named _children_): in our example, a child of vehicle could be the car and another child could be the bike: they have different properties and apply the functionalities in a different way. Both the vehicle, the car, and the bike may have instances: the car that you use everyady to go at work is different from the one of your friend even if they have the same exact properties, because they are two different concrete objects. Technically, those two cars are two _instances_ of the same _class_. To create an instance you have to call a function, which takes some argument such as the class, and other properties, and that will return the instance. To use `musiF`, you don't need to know a lot about objects, but while you search the web it is good to have a little of knowledge.

* a _dataframe_ is another way to represent information for computers. They are designed to be extremely efficient, even if sometimes some aspect of the information can get lost, and for this reason are used for data science problems. You can think about a _dataframe_ as to a table, with rows and columns. Usually, rows are _instances_ while columns are _properties_. In data science, these words often become _samples_ and _features_/_variables_. A typical operation is to select only certain columns (properties) or only certain rows (instances) to select subset of the data or to modify the data itself.
* don't be scared of using web search engines such as Google: searching the web in a proper way is one of the most important skills a programmer has!

### Main objects

When using `musif`, you will usually interface with two objects:
1. [`FeaturesExtractor()`](API/musif.extract.html#musif.extract.extract.FeaturesExtractor), which read scores in MusicXML and MuseScore format and computes a Dataframe containing all the extracted features. In the simplest case, each row represents a music score, while each column represents a feature.
2. [`DataProcessor()`](API/musif.process.html#musif.process.processor.DataProcessor), which takes the dataframe with all the features in it and post-processes it to clean, improve, and possibly modify some of the features.

These two objects take as input two different configurations that modify their behaviour. In other words, the function that create the instances of `FeaturesExtractor` and `DataProcessor` can accept a wide range of arguments.

But let's proceed step-by-step!

## Data

For starting, downaload one or more of the following datasets:

...

You should put the MusicXML data under the `data` directory aside to this notebook!